In [532]:
# setup
from IPython.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})

<h1>CMPS-2200 Introduction to Algorithms: Genome Sequencing</h1>



In this Jupyter Notebook, we will code Chapter 4 of the textbook Parallel Algorithms by Umut Acar. The point is to show the power of algorithms and give some examples before we dive into analysis and theory.

A **genome** is a sequence of the letters from the alphabet {'A', 'C', 'G', 'T'}. 

We want to learn the human genome.

The problem is that the human genome is very long (length ~3 billion), but we can only read snippets of length about 1000.
 

<h2>Breaking the problem into smaller pieces</h2>

To solve the problem, we use radiation to break the long genome sequence into smaller fragments randomly. Hopefully, each fragment will be small and can be sequenced. Then, we stitch the fragments together to recover the whole genome.

We illustrate this method on a genome sequence of length $N=1000$. First, we create a random string that represents the genome.

In [533]:
N=50 #length of the genome
alphabet = ['a','c','g','t'] #alphabet for genome
genome = ''.join(random.choices(alphabet,k=N)) #got this from here: https://www.geeksforgeeks.org/python-generate-random-string-of-given-length/


Next, we simulate the effect of radiation that breaks the string into num_fragments many fragments.

In [534]:
num_fragments = N //10 #global variable that gives the number of fragments we divide the genome into.
def create_fragments(genome,num_fragments):
    frag_indices = random.sample(range(len(genome)), num_fragments-1) #Chooses num_fragments many indicies. See https://docs.python.org/3/library/random.html
    frag_indices.sort()
    frag_indices.append(None)
    frag_indices=[0]+frag_indices #make sure to include the fragment that starts at 0.
    parts = {genome [frag_indices[i]:frag_indices[i+1]] for i in range(len(frag_indices)-1)}#Splits the genome at the frag_indicies. See https://stackoverflow.com/questions/10851445/splitting-a-string-by-list-of-indices 
    return parts
def check_num_fragments_works():
    print(genome)
    print(num_fragments)
    frags = create_fragments(genome, num_fragments)
    print(len(frags))
    print(frags)
check_num_fragments_works()

atgctgattagcggggccaagacaagttgcggcatttagggggatcttcc
5
5
{'ggggatctt', 'cc', 'aagacaagttgcggcatttag', 'atgctgattagcggggc', 'c'}


There are some strange things to notice:

    1. The fragments are stored as a set, not a list. We lose information about the order of the fragments.
    2. It is possible that some fragments could be duplicated, so there could be fewer than num_fragments many fragments.
    3. It is possible that some fragments could be too long to sequence.

We cannot hope to recover the genome from one set of fragments, because the fragmentation process forfits all information about the order of these fragments. The solution to this is to fragment the same genome multiple times so that fragments from different calls to create_fragments can overlap. We can use the overlap to stitch the fragments together.

To simulate this, let us apply the create_fragments method 3 times. The total fragments that we will see are the union of the fragments from several applications. Call the number of applications num_times_fragment

In [535]:
num_times_fragment = 3

fragments = create_fragments(genome, num_fragments)
for index in range(num_times_fragment -1):
    fragments = fragments.union(create_fragments(genome, num_fragments))
print(fragments)

{'ggccaagac', 'gacaagttgcg', 'atgctgat', 'aagtt', 'g', 'gcatttagggg', 'gggccaagacaagttgcggcatttagg', 'atgc', 'tagc', 'atgctgattagcgg', 'gatcttcc', 'tgattagcggggccaa', 'catttagggggatcttcc', 'gggatcttcc', 'gcgg'}


Hopefully, the fragments are small enough so that we can sequence them. Another benefit to dividing the genome into fragments is that all of the fragments can be sequenced in parallel (at the same time). Let as assume that we know these fragments. We must develop an algorithm to stitch these fragments back together.

<h2>Stitching the fragments back together</h2>

There are many ways to stitch the fragments back into a genome. For example, we could just concatenate all of the fragments. Since there are mulitple ways, we rely on a <i>heuristic</i> to guide us to a reasonable solution. Our heuristic is to choose the shortest genome that is compatible with these fragments. This is justified by Occam's Razor.

Now we have another problem: How do we stitch the fragments together into the shortest possible genome that has the fragments as substrings?

First, to simplify the problem, we can check the fragments to determine if one fragment is contained in another. If so, we discard the shorter fragment. We need a way to check if on substring is contained in another. We will give more details on this algorithm in the next lecture. For now, we use the python 'in' operator.

In [536]:
def remove_subfragments(fragments):
    fragments = {f for f in fragments if not any([f in other_frag and not f==other_frag for other_frag in fragments]) } #See https://realpython.com/python-string-contains-substring/ for this use of the 'in' operator to check substring membership.
    return fragments

def check_remove_subfragments_works():
    test_frags = {"a","ata", "attta", "gcgc", "gcagc", "ta", ""}
    assert remove_subfragments(test_frags) == {'ata', 'attta', 'gcagc', 'gcgc'}
check_remove_subfragments_works()

Now we apply the function and remove any fragments that are substrings of another fragment.

In [537]:
fragments = remove_subfragments(fragments)

Next, let us convert fragments from a set into a list, so that each fragment can be identified by its index in the list.

In [538]:
fragments = list(fragments)
print(fragments)

['gggccaagacaagttgcggcatttagg', 'atgctgattagcgg', 'tgattagcggggccaa', 'catttagggggatcttcc', 'gcatttagggg']


If we want the shortest possible genome compatible with these fragments, we may assume that each fragment only appears once in the genome. Thus, there is some order in which the fragments appear in the genome. If we knew this order, that we could stitch successive fragments together using the greatest possible overlap between successive pairs.

For example, suppose that we only have two fragments: ["agtgtg","tgtgc"] and that we know these fragments appear in this order. Then the genome containing these two fragments could be "agtgtgc," "agtgtgtgc", "agtgtgtgtgc", or "agtgtg*tgtgc" where * means any string can occur. Clearly, the first string is the shortest, because it provides the greatest overlap between the two fragments. 

We now write a function that calculates the overlap between two fragments.

In [539]:
def overlap(string1, string2):
    return max([index for index in range(min(len(string1), len(string2))) if string1[-index:]==string2[:index]], default =0 )

def check_overlap_works():
    str1 = "abba"
    str2 = "baab"
    str3 = "abbabbabb"
    str4 = "abbabbt"
    assert overlap(str1, str2) == 2
    assert overlap("", str2)==0
    assert overlap(str2,"")==0
    assert overlap(str3,str4) == 6

check_overlap_works()

It is possible to work directly with the overlap when deciding how to stitch the strings together. However, the textbook recommends using a `distance' function that also takes the length of the strings into account.

In [540]:
def distance_from_overlap(str1,str2):
    return len(str2) - overlap(str1,str2)

The distance defined above is not quite a distance, since it is not symmetric. However, it has the nice property that, if we know the order of the fragments, the length of the shortest genome constructed from them is just the sum of the distances from one fragment to the next, assuming that we start at an empty fragment. Let's add in the empty fragment so that this is the case.

In [541]:
fragments=[""] + fragments

We can calculate the distances between all pairs of fragments using a doubly-nested list. Later, we will call this object an "adjacency matrix."

In [542]:
all_pairs_of_distances = [[ distance_from_overlap(f1,f2) for f2 in fragments] for f1 in fragments]
print(fragments)
print(all_pairs_of_distances)
print([len(a) for a in fragments])

['', 'gggccaagacaagttgcggcatttagg', 'atgctgattagcgg', 'tgattagcggggccaa', 'catttagggggatcttcc', 'gcatttagggg']
[[0, 27, 14, 16, 18, 11], [0, 25, 14, 16, 10, 2], [0, 25, 14, 6, 18, 10], [0, 20, 13, 16, 18, 11], [0, 27, 14, 16, 17, 11], [0, 24, 14, 16, 8, 10]]
[0, 27, 14, 16, 18, 11]


(I am noticing that there tend to be repeated rows in the adjacency matrix. idk why)

<h2>The Travelling Salesman</h2>

We can visualize the fragments as <i>nodes</i> and the distances between them as <i>edges</i> labeled by the distance. For example, for the fragments "tagg" "catt" "gga" "gagtat" "tta" and "u" (the emptystring) we can draw the following diagram.

<img src="graph_from_fragments.png">


The drawing suggests a map and transforms our problem (shortest superstring) into a different problem (traveling salesman). The new problem is to start at the emptystring, then follow the arrows to visit each fragment exactly once, then return to the emptystring so that the total sum of distances is minimized.

The travelling salesman is an <i>NP-hard problem.</i> We will discuss this concept later, but for now this just means that the problem is considered difficult to solve. It is not that the problem is impossible to solve (uncomputable) but (to the best of our knoweledge) it would require too many resources (time, energy, dollars, etc) to solve the problem, even on moderately-sized instances.

For example, the <i>brute force</i> solution is always availible: we consider all possible len(fragments)! ways to order the fragments and choose the ordering with the smallest sum of distances. However, it is not computationally feasible to check all possible permutations.

There are better methods to find the optimal ordering of the fragments, but none of the known methods scale well as the number of fragments increases. There is a 1 million dollar bounty on this scaling problem, known as P=NP. In our case, we actually can solve the travelling salesman problem by using a library.

In [543]:
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming #https://pypi.org/project/python_tsp/

distance_matrix = np.array(all_pairs_of_distances)
permutation, distance = solve_tsp_dynamic_programming(distance_matrix)
print(permutation, distance)

[0, 2, 3, 1, 5, 4] 50


The output of the cell above tells us how to re-order the fragments to produce the smallest genome that could have created them. Next we need to apply this order.

In [544]:
ordered_fragments = [fragments[permutation[index]] for index in range(len(fragments))]
#Let's show the re-ordered fragments and the overlaps between adjacent fragments
if len(fragments)<25: #if we run this on a smaller set of fragments, let's inspect by eye that this re-ordering is good.
    print("ordered fragments: ", ordered_fragments)
    print("overlaps between succesive pairs")
    print([overlap(ordered_fragments[i], ordered_fragments[i+1])for i in range(len(ordered_fragments)-1)])

ordered fragments:  ['', 'atgctgattagcgg', 'tgattagcggggccaa', 'gggccaagacaagttgcggcatttagg', 'gcatttagggg', 'catttagggggatcttcc']
overlaps between succesive pairs
[0, 10, 7, 9, 10]


In [545]:
def stitch_fragments(frags): #assume that frags is a non-empty list of fragments, in the order that you want to stitch them.
    trimmed_frags = [frags[0]]
    for i in range(len(frags)-1):
        trimmed_frags.append(frags[i+1][overlap(frags[i],frags[i+1]):] )
    return ''.join(trimmed_frags)

def test_stitch_fragments():
    assert stitch_fragments(['', 'gtggcggttcatgcagactaattggccattcca', 'gactaattggccattccaaa', 'aatg', 'tgagtggtgggaagg']) =="gtggcggttcatgcagactaattggccattccaaatgagtggtgggaagg"
    #Overlaps between succesive fragments are [0, 18, 2, 2]
test_stitch_fragments()



In [546]:
reconstructed_genome = stitch_fragments(ordered_fragments)
assert reconstructed_genome == genome

The assertion above usually passes, but not quite every time.

We have succeeded in reconstructing the genome, but we need to answer some serious questions: Can we count on this to work?

<h2>Scalability</h2>

Algorithms are important for working on big problems. Small problems can be solved inefficiently, since they require little energy.

The travelling salesman problem cannot be solved for massive parameters, like N=3 billion. However, it is too soon to give up. We turn to another heuristic that we will see in more detail later in the class. This is the concept of a <i>greedy algorithm</i>.

A greedy algorithm reduces optimization problems (like traveling salesman) into a series of simple steps. At each step, the algorithm chooses the best option. The resulting solution is not always optimal, but sometimes it's good enough.

In [550]:
def greedy_tour(distance_matrix): #Assume distance-matrix is a doubly nested array so that distance_matrix[i,j] is the distance from the ith fragment to the jth fragment.
    current_index = 0
    tour = [0]
    unvisited_vertices = list(range(len(distance_matrix)))[1:] #we start at vertex 0
    while len(unvisited_vertices)>0:

        current_distance=math.inf        
        for j in unvisited_vertices:
            if distance_matrix[current_index][j]<current_distance:
                next_index = j
                current_distance= distance_matrix[current_index][j]
        current_index = next_index
        tour.append(current_index)
        unvisited_vertices.remove(current_index)
    return(tour) #Returns a sequence of indices, starting at index 0, that includes the index of each node exactly once.

def test_greedy_tour():
    A=[[0,2,5,3,1],[0,6,2,1,3],[0,9,9,8,11], [0,1,7,7,6,2], [0,4,3,1,2]]
    assert greedy_tour(A) == [0, 4, 3, 1, 2]

test_greedy_tour()


We use the greedy tour to order the fragments before stitching them together.

In [551]:
permutation = greedy_tour(all_pairs_of_distances)
ordered_fragments = [fragments[permutation[index]] for index in range(len(fragments))]

Finally, we stitch these ordered fragments back together.

greedily_recovered_genome = 